**Prompt**: Use the pure pytorch linear modules etc.,. - high level primitives and build 3 layer neural network.

In [ ]:
import torch
import torch.nn as nn

# Creating Nonlinear Equation, Generating Data

In [ ]:
# Generating data of 3 dim input and 2 dim output
n, d = 500, 3
x = torch.FloatTensor(n, d).uniform_(-1, 1)
weights_true = torch.tensor([[5,1,5],[1,2,1]]).float()
weights_true = torch.transpose(weights_true,0,1)
bias_true = torch.tensor([1,2])
y_true = torch.mm(x**2,weights_true) + torch.mm(x,weights_true) + bias_true
print(f'x: {x.shape}, weights: {weights_true.shape}, bias: {bias_true.shape}, y: {y_true.shape}')

x: torch.Size([500, 3]), weights: torch.Size([3, 2]), bias: torch.Size([2]), y: torch.Size([500, 2])


# Building 3 layer NN with Pytorch using linear modules

In [ ]:
# Custom Model using nn.Linear modules
class TorchModel(nn.Module):
  def __init__(self, input_dim, num_hidden):
    super().__init__()
    self.linear1 = nn.Linear(input_dim, num_hidden)
    self.relu1 = nn.ReLU()
    self.linear2 = nn.Linear(num_hidden, 12)
    self.relu2 = nn.ReLU()
    self.linear3 = nn.Linear(12, 2)
  
  def forward(self, x):
    l1 = self.linear1(x)
    r1 = self.relu1(l1)
    l2 = self.linear2(r1)
    r2 = self.relu2(l2)
    l3 = self.linear3(r2)
    return l3

# Training/Testing the Model

In [ ]:
# Now we run the training loop
from typing import Callable

def torch_fit(x, y, model: Callable, loss: Callable, lr: float, num_epochs: int):
  optimizer = torch.optim.SGD(model.parameters(), lr=lr)
  for epoch in range(num_epochs):
    optimizer.zero_grad() # Initialize gradient as zero
    y_pred = model(x) # Forward Pass
    loss_value = loss(y_pred, y) # Compute loss with MSE
    if epoch % 50 == 0:
      print(f'Epoch {epoch}, loss {loss_value}')
    loss_value.backward() # Use autogradient to compute backward pass
    optimizer.step() # Update weights 

loss = nn.MSELoss()
model = TorchModel(d, 22)
torch_fit(x, y_true, model=model, loss=loss, lr=0.035, num_epochs=1000)

Epoch 0, loss 25.69439125061035
Epoch 50, loss 1.697373390197754
Epoch 100, loss 0.5532752275466919
Epoch 150, loss 0.2408604770898819
Epoch 200, loss 0.8408664464950562
Epoch 250, loss 0.156325101852417
Epoch 300, loss 0.13476543128490448
Epoch 350, loss 0.2834080457687378
Epoch 400, loss 0.12009493261575699
Epoch 450, loss 0.1222260519862175
Epoch 500, loss 0.16142800450325012
Epoch 550, loss 0.10993946343660355
Epoch 600, loss 0.12285434454679489
Epoch 650, loss 0.11488842964172363
Epoch 700, loss 0.09676457941532135
Epoch 750, loss 0.10655902326107025
Epoch 800, loss 0.10887341946363449
Epoch 850, loss 0.09654075652360916
Epoch 900, loss 0.09911637753248215
Epoch 950, loss 0.09871859848499298


# Plotting Training Data and Prediction

In [ ]:
def plot_intereactive_3d(x, y, y_pred=None):
  import plotly.graph_objects as go

  fig = go.Figure()
  fig.add_trace(go.Scatter3d(x = x[:,0],
                    y = x[:,1],
                    z = y.reshape([-1]),
                    opacity=0.5, mode='markers', name='Underlying Function'
                    ))
 
  if y_pred is not None:
    fig.add_trace(go.Scatter3d(x = x[:,0],
                   y = x[:,1],
                   z = y_pred.reshape([-1]),
                   opacity=0.5, mode='markers', name='Predicted Function'
                  ))
    
  fig.update_layout(scene = dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Y'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))
  fig.show()

In [ ]:
from sklearn.manifold import TSNE
X_reduced = TSNE(n_components=2).fit_transform(x)
y_true_reduced = TSNE(n_components=1).fit_transform(y_true)
y_pred_reduced = TSNE(n_components=1).fit_transform(model(x).detach())
print(f'X_reduced: {X_reduced.shape}, y_true_reduced: {y_true_reduced.shape}, y_pred_reduced: {y_pred_reduced.shape}')

plot_intereactive_3d(X_reduced,y_true_reduced,y_pred_reduced)

X_reduced: (500, 2), y_true_reduced: (500, 1), y_pred_reduced: (500, 1)
